In [1]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf

import os
import h5py
import json
import gc
import io

from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, AvgPool2D, Flatten, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import Callback


2023-12-21 09:07:05.888883: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-21 09:07:07.531775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 09:07:07.531811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 09:07:07.714387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 09:07:08.091548: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf

import os
import h5py
import json
import gc
import io

from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, AvgPool2D, Flatten, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import Callback


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress most of the logs
tf.get_logger().setLevel('WARNING')  # Adjust logging level

# Set the GPU memory growth option
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to allocate only the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Enable memory growth to allocate GPU memory on an as-needed basis
        tf.config.experimental.set_memory_growth(gpus[0], True)
        # Set the = activationdesired memory limit (in MB)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20480)]  # 20GB
        )
    except RuntimeError as e:
        print(e)

# Load data

# Load one image to get the input shape
with h5py.File('image.h5', 'r') as h5file:
    one_file = h5file['images'][0:1]  # Load the first image

y = np.load('styles.npy', allow_pickle=True)
# Not going to load X yet. because it is too big.
# We are going to load X batch by batch when model.fit.

le = LabelEncoder()
y = le.fit_transform(y)

# Assuming y contains integer labels
y = to_categorical(y, num_classes=7)

# Assuming total number of images
num_images = len(y)  # or len(combined_df)
indices = np.arange(num_images)

# Split indices
indices_train, indices_temp, y_train, y_temp = train_test_split(indices,y, test_size=0.2, random_state=1, stratify=y)
indices_val, indices_test, y_val, y_test = train_test_split(indices_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

np.save('indices_test.npy', np.array(indices_test))

# yield each batch
def data_generator(h5_path, indices, styles, batch_size):
    with h5py.File(h5_path, 'r') as h5file:
        while True:
            np.random.shuffle(indices)
            for i in range(0, len(indices), batch_size):
                batch_indices = indices[i:i + batch_size]
                # Sort the batch_indices to meet HDF5's requirements
                sorted_batch_indices = np.sort(batch_indices)
                batch_images = h5file['images'][sorted_batch_indices]
                batch_styles = styles[sorted_batch_indices]
                yield batch_images, batch_styles


# Create generators
batch_size = 8  # Define your batch size
train_generator = data_generator('image.h5', indices_train, y, batch_size)
val_generator = data_generator('image.h5', indices_val, y, batch_size)
test_generator = data_generator('image.h5', indices_test, y, batch_size)

steps_per_epoch = len(indices_train) // batch_size
validation_steps = len(indices_test) // batch_size

2023-12-21 09:09:08.760706: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 09:09:10.015915: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 09:09:10.016036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
model = tf.keras.models.load_model('Conv2D_handmade_model.h5')

In [9]:
def data_generator_predict(h5_path, indices, batch_size):
    with h5py.File(h5_path, 'r') as h5file:
        # No shuffling for prediction
        for i in range(0, len(indices), batch_size):
            batch_indices = indices[i:i + batch_size]
            sorted_batch_indices = np.sort(batch_indices)
            batch_images = h5file['images'][sorted_batch_indices]
            yield batch_images

# Create test generator for prediction
test_generator_predict = data_generator_predict('image.h5', indices_test, batch_size)

# Calculate steps for the test generator
test_steps = len(indices_test) // batch_size

# Predict
y_pred = model.predict(test_generator_predict, steps=test_steps)

109/109 [==============================] - 164s 2s/step


In [10]:
y_pred_labels = np.argmax(y_pred, axis=1)

y_test_labels = np.argmax(y_test, axis=1)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test_labels, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, y_pred_labels))
print("Classification Report:\n", classification_report(y_test_labels, y_pred_labels))

Accuracy: 0.536697247706422
Confusion Matrix:
 [[ 12   5   6   0  84   0   3]
 [  7   3   7   2  43   0   2]
 [  5   3   4   0  30   0   2]
 [  0   1   0   0  13   0   0]
 [ 54  51  31  15 449   4  11]
 [  0   1   0   0   4   0   0]
 [  1   1   4   0  13   1   0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.15      0.11      0.13       110
           1       0.05      0.05      0.05        64
           2       0.08      0.09      0.08        44
           3       0.00      0.00      0.00        14
           4       0.71      0.73      0.72       615
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00        20

    accuracy                           0.54       872
   macro avg       0.14      0.14      0.14       872
weighted avg       0.52      0.54      0.53       872



In [12]:
# Assuming le is your LabelEncoder object and y_pred_labels are your predicted labels
label_names = le.inverse_transform([0,1,2,3,4,5,6])

# Now label_names will have the original string names of the labels
print(label_names)

['Classic' 'Contemporary' 'Country' 'Minimalism' 'Modern' 'Unique' 'Urban']


In [15]:
import matplotlib.pyplot as plt
import json

# Load the history from the JSON file
with open('history.json', 'r') as f:
    history = json.load(f)


# Assuming 'history' is the variable returned by model.fit()

# Plot for loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot for accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


JSONDecodeError: Expecting value: line 1 column 1262 (char 1261)